In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore")

from scipy import stats

import env
import wrangle as w
import explore as e
import modeling as m

In [13]:
# acquire & clean the data
df = w.wrangle_telco_data()

# split the data into train, validate, and test subsets
train, validate, test = w.split_my_data(df)

In [14]:
train.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,tech_support,contract_type_id,payment_type_id,monthly_charges,total_charges,churn
0,Male,0,No,No,48,Yes,Yes,2,No,3,3,108.10,5067.45,No
1,Male,0,Yes,Yes,63,Yes,Yes,3,No internet service,3,2,25.25,1573.05,No
2,Male,0,Yes,No,52,Yes,No,1,No,2,2,75.40,3865.45,No
3,Male,0,Yes,No,72,Yes,Yes,2,Yes,3,4,109.75,7758.90,No
4,Male,1,Yes,No,72,No,No phone service,1,Yes,3,4,64.45,4641.10,No


In [15]:
keep_cols = ['churn','monthly_charges','senior_citizen','tech_support']

train = train[keep_cols]
validate = validate[keep_cols]
test = test[keep_cols]
    
    # Split data into predicting variables (X) and target variable (y) and reset the index for each dataframe
train_X = train.drop(columns='churn').reset_index(drop=True)
train_y = train[['churn']].reset_index(drop=True)

validate_X = validate.drop(columns='churn').reset_index(drop=True)
validate_y = validate[['churn']].reset_index(drop=True)

test_X = test.drop(columns='churn').reset_index(drop=True)
test_y = test[['churn']].reset_index(drop=True)
    
    # manual encoding
train_X['senior_citizen'] = train_X.senior_citizen.apply(lambda value: 1 if value == 'Yes' else 0)
train_X['tech_support'] = train_X.tech_support.apply(lambda value: 1 if value == 'Yes' else 0)
    
validate_X['senior_citizen'] = train_X.senior_citizen.apply(lambda value: 1 if value == 'Yes' else 0)
validate_X['tech_support'] = train_X.tech_support.apply(lambda value: 1 if value == 'Yes' else 0)
    
test_X['senior_citizen'] = train_X.senior_citizen.apply(lambda value: 1 if value == 'Yes' else 0)
test_X['tech_support'] = train_X.tech_support.apply(lambda value: 1 if value == 'Yes' else 0)

    # Change target column to show values as upset or non-upset
train_y['churn'] = train_y.churn.apply(lambda value: "churn" if value == 'Yes' else "no-churn")
validate_y['churn'] = validate_y.churn.apply(lambda value: "churn" if value == 'Yes' else "no-churn")
test_y['churn'] = test_y.churn.apply(lambda value: "churn" if value == 'Yes' else "no-churn")

In [16]:
# return random forest results
m.ran_forest_acc(train_X, validate_X, train_y, validate_y)

Random Forest accuracy on train data: 0.7421894843789688
Random Forest accuracy on validate data: 0.7209715639810427


In [17]:
rf = RandomForestClassifier(max_depth=5, random_state= 903)
rf.fit(train_X,train_y)

RandomForestClassifier(max_depth=5, random_state=903)

In [18]:
print(f'Random Forest accuracy on test data: {rf.score(test_X, test_y)}')

Random Forest accuracy on test data: 0.7213930348258707


In [19]:
predictions = rf.predict(test_X)

In [20]:
predictions

array(['no-churn', 'no-churn', 'no-churn', ..., 'no-churn', 'no-churn',
       'no-churn'], dtype=object)

In [21]:
probabilities = rf.predict_proba(test_X)

In [35]:
probabilities

array([[0.17052797, 0.82947203],
       [0.47324659, 0.52675341],
       [0.16669833, 0.83330167],
       ...,
       [0.47094925, 0.52905075],
       [0.47094925, 0.52905075],
       [0.09180803, 0.90819197]])

In [2]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [6]:
 # read the SQL query into a dataframe
df2 = pd.read_sql('SELECT * FROM customers', get_connection('telco_churn'))

        # drop redundancies and columns not needed for this exploration
df2 = df2.drop(columns=['online_security','online_backup',\
                              'device_protection','streaming_tv',\
                              'streaming_movies','paperless_billing'])

        # modify total_charges column by removing spaces and convert to float
df2['total_charges'] = df2['total_charges'].str.replace(' ', '')
df2['total_charges'] = pd.to_numeric(df2['total_charges'])

# only 11 missing values from 1 column. I will drop these rows
df2 = df2.dropna()

In [7]:
df2

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,tech_support,contract_type_id,payment_type_id,monthly_charges,total_charges,churn
0,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,1,Yes,2,2,65.60,593.30,No
1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,1,2,59.90,542.40,No
2,0004-TLHLJ,Male,0,No,No,4,Yes,No,2,No,1,1,73.90,280.85,Yes
3,0011-IGKFF,Male,1,Yes,No,13,Yes,No,2,No,1,1,98.00,1237.85,Yes
4,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,2,Yes,1,2,83.90,267.40,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9987-LUTYD,Female,0,No,No,13,Yes,No,1,Yes,2,2,55.15,742.90,No
7039,9992-RRAMN,Male,0,Yes,No,22,Yes,Yes,2,No,1,1,85.10,1873.70,Yes
7040,9992-UJOEL,Male,0,No,No,2,Yes,No,1,No,1,2,50.30,92.75,No
7041,9993-LHIEB,Male,0,Yes,Yes,67,Yes,No,1,Yes,3,2,67.85,4627.65,No


In [8]:
train2, validate2, test2 = w.split_my_data(df2)

In [10]:
test2 = test2.customer_id

In [11]:
test2

0       8165-CBKXO
1       1302-UHBDD
2       7139-JZFVG
3       3207-OYBWH
4       2272-QAGFO
           ...    
1402    5577-OTWWW
1403    1217-VASWC
1404    6257-DTAYD
1405    6728-DKUCO
1406    6734-CKRSM
Name: customer_id, Length: 1407, dtype: object

In [34]:
predictions.shape

(1407,)

In [33]:
probabilities.shape

(1407, 2)

In [41]:
dfprob = pd.DataFrame(data = probabilities)

In [51]:
dfpred = pd.DataFrame(data = predictions)

In [53]:
predicitiondf = pd.concat([test2, dfprob, dfpred], axis = 1)

In [54]:
predicitiondf.to_csv('predictions.csv')